# P5 - Vehicle Detection - Test new function

# File sections:
* import section
* general utilities
* load the training data and data exporation
* HOG parameters selection by visualization
* Spacial parameters selection by visualization
* Color Hist parmeters selection

## imports for the whole project

In [236]:
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
import pickle
import os
import matplotlib.image as mpimg
from sklearn.preprocessing import StandardScaler
from skimage.feature import hog
from sklearn.model_selection import train_test_split
from random import randint
import time
from sklearn.svm import LinearSVC

%matplotlib inline

### imports for the whole project - end

## General Utility Functions - not directed related to the project

In [237]:
# Read Image
def readImage(imageFileName, cFormat='RGB'): # cFormat, BGR, CV, RGB, MATPLOT. BGR and CV are the same. RGB and MATPLOT are the same
    img_bgr_cv2 = cv2.imread(imageFileName) # mpimg.imread(img_fname)
    if (cFormat == 'BGR' or cFormat == 'CV'):
        return img_bgr_cv2
    elif (cFormat == 'RGB' or cFormat == 'MATPLOT'):
        img_rgb_mtplt = cv2.cvtColor(img_bgr_cv2, cv2.COLOR_BGR2RGB)
        return img_rgb_mtplt  # This is the matplot fomrat
    return img_bgr_cv2

def readImages(dir, pattern = '*[.png][.jpg][.jepg]', cFormat='RGB'):
    # read all images with the given patterns (extensions) in the sub-directories
    # Example of a pattern: pattern = '*[.png][.jpg]'

    """
    Returns an image list with the image contained on the directory `dir` matching the `pattern`.
    pattern
    """
    images = []
    for imageFileName in glob.iglob(dir + '**/' + pattern, recursive=True):
        images.append(readImage(imageFileName, cFormat))
    return images

def readImagesFileNames(dir, pattern = '*[.png][.jpg][.jepg]', cFormat='RGB'):
    # read all images with the given patterns (extensions) in the sub-directories
    # Example of a pattern: pattern = '*[.png][.jpg]'

    """
    Returns 2 lists of image names list. File name and full path name

    """
    imagesFileNames = []
    imagesFullFileNames = []
    for imageFullFileName in glob.iglob(dir + '**/' + pattern, recursive=True):
        imagesFullFileNames.append(imageFullFileName)
        imagesFileNames.append(os.path.basename(imageFullFileName))
    return imagesFileNames, imagesFullFileNames

def display_save_images_single_line(images, folder, titles, figTitel='', figsize=(6, 3), gray=False):
    plt.figure(figsize=figsize)
    for i in range (len(images)):
        plt.subplot(1, len(titles), i+1)
        plt.title(titles[i])
#        plt.axis('off')
        if gray == True:
            plt.imshow(images[i], cmap='gray')
        else:
            plt.imshow(images[i]) # , cmap='binary'
        plt.tight_layout(pad=0.50)
    plt.tight_layout(pad=0.50)
#    plt.xlabel(figTitel) #, fontdict=font)
    plt.suptitle(figTitel, fontsize=16)
    plt.subplots_adjust(left=0, wspace=0, top=0.8)
    if not os.path.exists(folder): #if output dir doesn't exists, create it
        os.makedirs(folder)
        plt.savefig(os.path.join(folder, figTitel))
    plt.show()

def display_save_2_images(images, titles, display_flag='TRUE', image_file_full_name='./image.png', file_prefix="", file_suffix="", output_directory='./output_images/'):
    if (display_flag == 'TRUE'):
        plt.figure(figsize=(6, 3))
        plt.subplot(1, 2, 1)
        plt.title(titles[0])
        plt.axis('off')
        plt.imshow(images[0], cmap='binary')
        plt.subplot(1, 2, 2)
        plt.title(titles[1])
        plt.axis('off')
        plt.imshow(images[1], cmap='binary')
        plt.tight_layout(pad=0.50)
        plt.tight_layout(pad=0.50)
        plt.suptitle(new_file_name)
        base=os.path.basename(image_file_full_name)
        (file_name,ext) = os.path.splitext(base)
        new_file_name = file_prefix + file_name + file_suffix + ext
        image_file_full_name_newdir = os.path.join(output_directory, new_file_name)
        print ('Saving: ' , image_file_full_name, new_file_name, image_file_full_name_newdir)
        if not os.path.exists(output_directory): #if output dir doesn't exists, create it
            os.makedirs(output_directory)
        plt.savefig(image_file_full_name_newdir)
        plt.show()
    return

### General Utility Functions - end

## Load the training data and data exporation

### Read the training images

In [238]:
vehiclesFN, vehiclesFFN = readImagesFileNames('./training_images/vehicles/vehicles/', '*.png')
non_vehiclesFN, non_vehiclesFFN = readImagesFileNames('./training_images/non-vehicles/non-vehicles/', '*.png')
print ('Number of Vehicle images = ' + str(len(vehiclesFN)))
print ('Number of non-Vehicle images = ' + str(len(non_vehiclesFFN)))

Number of Vehicle images = 8791
Number of non-Vehicle images = 8968


## Parameters definition - classes

# Value object to hold all feature extraction parameters.

In [239]:
class HogParameters():
    def __init__(self):
        # HOG parameters
        self.orient = 8
        self.pix_per_cell = 8
        self.cell_per_block = 2
        self.vis= False

class FeaturesParameters():
    def __init__(self):
        # consider different channels from differnet colors
        self.combined_channels = [['YCrCb',[0,1,2]]] #,['HLS',[0]] # ['RGB',[0]], ['YCrCb',[0,2]],['HSV',[0]],['LUV',[0,1]]] # 6 channels! instead of 3'RGB' #'YCrCb' [['YCrCb',[0,1,2]]] #
        self.color_space = 'YCrCb' #[('RGB',0),('YCrCb',1),('HSV',0),('LUV',1),('LUV',1)] #'RGB' #'YCrCb'
        # Bin spatial parameters
        self.spatial_size = (16, 16)
        # Histogram parameters
        self.hist_bins = 32
        self.hist_range = (0, 255)
        # HOG parameters
        self.hog_params=HogParameters()
        
features_params = FeaturesParameters()

### Parameters definition - end

## HOG parameters selection - HOG visualization

### Note: Original image and color converted image are intentially not shown in order to properly detect the best channel!

In [240]:
# Define a function to return HOG features and visualization
def color_convert (image, color_space='RGB'):
    if color_space != 'RGB':
        if color_space == 'HSV':
            feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
        elif color_space == 'LUV':
            feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2LUV)
        elif color_space == 'HLS':
            feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2HLS)
        elif color_space == 'YUV':
            feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2YUV)
        elif color_space == 'YCrCb':
            feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2YCrCb)
        else:
            print("Error undefined color sapace: ", color_space)
    else: 
        feature_image = np.copy(image)      
    return feature_image

def color_convert_combined (image, combined_channels=[['RGB',[0,1,2]]]):
    number_of_channels=0
    for color_sys in range (len(combined_channels)):
        feature_image = color_convert(image, combined_channels[color_sys][0])
        for channel in range (len(combined_channels[color_sys][1])):
            if number_of_channels==0:
                all_ch_img = feature_image[:,:,combined_channels[color_sys][1][channel]]
                all_ch_img = np.expand_dims(all_ch_img, axis=2)
            else:
                chanel = combined_channels[color_sys][1][channel]
                chanel_image = feature_image[:,:,combined_channels[color_sys][1][channel]]
                all_ch_img = np.insert(all_ch_img,0,chanel_image, axis=2)
            number_of_channels+=1
    return number_of_channels, all_ch_img

def get_hog_channel_features(feature_image, p, feature_vec):
    return (hog(feature_image, 
                            orientations=p.orient, pixels_per_cell=(p.pix_per_cell,p.pix_per_cell), 
                            cells_per_block=(p.cell_per_block, p.cell_per_block), visualise=False, transform_sqrt=True, feature_vector=feature_vec))

def get_hog_features(feature_image, p):
    hog_features = []
    for channel in range(feature_image.shape[2]):
        hog_features.append(get_hog_channel_features(feature_image[:,:,channel], p, feature_vec=True))
    hog_features = np.ravel(hog_features)        
    return hog_features

# Define a function to compute binned color features  
def bin_spatial(img, size=(32, 32)):
    colors = cv2.resize(img[:,:,img.shape[2]-1], size).ravel()
    for channel in range(img.shape[2]-1):
        nextColor = cv2.resize(img[:,:,img.shape[2]-channel-2], size).ravel()
        colors = np.hstack((colors, nextColor))
    return colors

# Define a function to compute color histogram features 
# NEED TO CHANGE bins_range if reading .png files with mpimg!

def color_hist(img, nbins=32, bins_range=(0, 256)):
    # Compute the histogram of the color channels separately
    hists = np.histogram(img[:,:,img.shape[2]-1], bins=nbins)[0]
    for channel in range(img.shape[2]-1):
            nextHist = np.histogram(img[:,:,img.shape[2]-channel-2], bins=nbins)[0] # , range=bins_range
            hists = np.concatenate(([hists, nextHist]))
    return hists

In [241]:
# Define a function to extract features from a list of images
# Have this function call bin_spatial() and color_hist()

# Parameters: color_space: 
# num of orientation bins (how many orientation. Normally 9)
# grid of the cells: how many cells in the image 
#cell size: pixcels / cel
# adding overlaps between cells
# block normalizarion
def extract_features(image, params):

    number_of_channels, feature_image = color_convert_combined(image, params.combined_channels)
    image_features = []

    # Apply bin_spatial() to get spatial color features
    spatial_features = bin_spatial(feature_image, size=params.spatial_size)

    # Apply color_hist()
    hist_features = color_hist(feature_image, nbins=params.hist_bins, bins_range=params.hist_range)

    # Apply hog() to get histogram orientation gradient features
    hog_features = get_hog_features(feature_image, params.hog_params)
    tmp_features = np.concatenate((spatial_features, hist_features, hog_features))
    
    return tmp_features

def extract_features_images(images, params):
    # Create a list to append feature vectors to
    features = []
    for img in images:
        imgFeatures = extract_features(img, params)
        features.append(imgFeatures)
    return features

In [242]:
combined_channels = features_params.combined_channels
def extract_features_old(imgs, color_space='RGB', spatial_size=(32, 32),
                        hist_bins=32, orient=9, 
                        pix_per_cell=8, cell_per_block=2, hog_channel=0,
                        spatial_feat=True, hist_feat=True, hog_feat=True):

    # Create a list to append feature vectors to
    features = []
    # Iterate through the list of images
    for image in imgs:
        file_features = []
        # Read in each one by one
        # apply color conversion if other than 'RGB''ALL'
        
        if color_space != 'RGB':
            if color_space == 'HSV':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
            elif color_space == 'LUV':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2LUV)
            elif color_space == 'HLS':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2HLS)
            elif color_space == 'YUV':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2YUV)
            elif color_space == 'YCrCb':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2YCrCb)
            else:
                print(color_space)
        else: feature_image = np.copy(image)      

        if spatial_feat == True:
            spatial_features = bin_spatial_old(feature_image, size=spatial_size)
            file_features.append(spatial_features)
        if hist_feat == True:
            # Apply color_hist()
            hist_features = color_hist_old(feature_image, nbins=hist_bins)
            file_features.append(hist_features)
        if hog_feat == True:
        # Call get_hog_features() with vis=False, feature_vec=True
            if hog_channel == 'ALL':
                
                print ('ALL Feautres')
                hog_features = []
                for channel in range(feature_image.shape[2]):
                    hog_features.append(get_hog_features_old(feature_image[:,:,channel], 
                                        orient, pix_per_cell, cell_per_block, 
                                        vis=False, feature_vec=True))
                hog_features = np.ravel(hog_features)        
            else:
                hog_features = get_hog_features_old(feature_image[:,:,hog_channel], orient, 
                            pix_per_cell, cell_per_block, vis=False, feature_vec=True)
            # Append the new feature vector to the features list
            file_features.append(hog_features)
        features.append(np.concatenate(file_features))
    # Return list of feature vectors
    return features
 

In [243]:
def get_hog_features_old(img, orient, pix_per_cell, cell_per_block, 
                        vis=False, feature_vec=True):
    # Call with two outputs if vis==True
    if vis == True:
        features, hog_image = hog(img, orientations=orient, 
                                  pixels_per_cell=(pix_per_cell, pix_per_cell),
                                  cells_per_block=(cell_per_block, cell_per_block), 
                                  transform_sqrt=True, 
                                  visualise=vis, feature_vector=feature_vec)
        return features, hog_image
    # Otherwise call with one output
    else:      
        features = hog(img, orientations=orient, 
                       pixels_per_cell=(pix_per_cell, pix_per_cell),
                       cells_per_block=(cell_per_block, cell_per_block), 
                       transform_sqrt=True, 
                       visualise=vis, feature_vector=feature_vec)
        return features

# def bin_spatial(img, size=(32, 32)):
#     # Use cv2.resize().ravel() to create the feature vector
#     features = cv2.resize(img, size).ravel() 
#     # Return the feature vector
#     return features
def bin_spatial_old(img, size=(32, 32)):
    color1 = cv2.resize(img[:,:,0], size).ravel()
    color2 = cv2.resize(img[:,:,1], size).ravel()
    color3 = cv2.resize(img[:,:,2], size).ravel()
    return np.hstack((color1, color2, color3))


def color_hist_old(img, nbins=32):    #bins_range=(0, 256)
    # Compute the histogram of the color channels separately
    channel1_hist = np.histogram(img[:,:,0], bins=nbins)
    channel2_hist = np.histogram(img[:,:,1], bins=nbins)
    channel3_hist = np.histogram(img[:,:,2], bins=nbins)
    # Concatenate the histograms into a single feature vector
    hist_features = np.concatenate((channel1_hist[0], channel2_hist[0], channel3_hist[0]))
    # Return the individual histograms, bin_centers and feature vector
    return hist_features

In [244]:
# Test Colors
tst_colors=['RGB', 'HSV', 'LUV', 'HLS', 'YUV', 'YCrCb']
test_vehicles_num = [7] #, 100, 30, 77, 55]
test_vehicles = []

for veh_num in test_vehicles_num:
    test_vehicles.append(readImage(vehiclesFFN[veh_num], 'RGB'))

# [['YCrCb',[0,1]],'HLS',[0]]
# ['RGB',[0,1,2]]
def test_spacial():
    test_features_params = FeaturesParameters()
    num_of_errors = 0

    for color in tst_colors:
        test_features_params.combined_channels = [[color,[0,1,2]]]
        test_features_params.color_space = color
        features = extract_features_images(test_vehicles, test_features_params)
        Oldfeatures = extract_features_old(test_vehicles, test_features_params.color_space, test_features_params.spatial_size, 
                                           test_features_params.hist_bins, test_features_params.hog_params.orient,
                                           test_features_params.hog_params.pix_per_cell, test_features_params.hog_params.cell_per_block,
                                           'ALL', True, True, True)
    print(len(features), len(Oldfeatures))
    print(len(features[0]), len(Oldfeatures[0]))
#    print(len(features[1]), len(Oldfeatures[1]))
#    print(len(features[2]), len(Oldfeatures[2]))
#    print(len(features[3]), len(Oldfeatures[3]))
#    print(len(features[4]), len(Oldfeatures[4]))
    features[0].sort()
    Oldfeatures[0].sort()
    for i, j in zip(features[0], Oldfeatures[0]):
        if (i!=j):
            print (i, ",", j)
            num_of_errors+=1
#    print(Oldfeatures[0])
#     print(features.shape, Oldfeatures.shape)
#     if (features != Oldfeatures):
    return num_of_errors

print ("Num of errors: ", test_spacial())

ALL Feautres
ALL Feautres
ALL Feautres
ALL Feautres
ALL Feautres
ALL Feautres
1 1
5568 5568
Num of errors:  0


/home/carkyo1080-1/anaconda3/envs/carnd-term1/lib/python3.5/site-packages/skimage/feature/_hog.py:119: skimage_deprecation: Default value of `block_norm`==`L1` is deprecated and will be changed to `L2-Hys` in v0.15
  'be changed to `L2-Hys` in v0.15', skimage_deprecation)
